# Problem Statement

In [428]:
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [429]:
A = pd.read_csv("D:/EtlHiveDs/Project/Project1/training_set.csv")

In [430]:
A["MSSubClass"] = A["MSSubClass"].map(str)
A["MoSold"] = A["MoSold"].map(str)

In [431]:
A["Alley"] = A["Alley"].fillna("NoAlley")
A["BsmtQual"] = A["BsmtQual"].fillna("NoBasement")
A["BsmtCond"] = A["BsmtCond"].fillna("NoBasement")
A["BsmtExposure"] = A["BsmtExposure"].fillna("NoBasement")
A["BsmtFinType1"] = A["BsmtFinType1"].fillna("NoBasement")
A["BsmtFinType2"] = A["BsmtFinType2"].fillna("NoBasement")
A["FireplaceQu"] = A["FireplaceQu"].fillna("NoFireplace")
A["GarageType"] = A["GarageType"].fillna("NoGarage")
A["GarageFinish"] = A["GarageFinish"].fillna("NoGarage")
A["GarageQual"] = A["GarageQual"].fillna("NoGarage")
A["GarageCond"] = A["GarageCond"].fillna("NoGarage")
A["PoolQC"] = A["PoolQC"].fillna("NoPool")
A["Fence"] = A["Fence"].fillna("NoFence")
A["MiscFeature"] = A["MiscFeature"].fillna("None")

In [432]:
A.OverallQual.replace({
    10:"Very Excellent",
    9:"Excellent",
    8:"Very Good",
    7:"Good",
    6:"Above Average",
    5:"Average",
    4:"Below Average",
    3:"Fair",
    2:"Poor",
    1:"Very Poor"},inplace=True)

In [433]:
A.OverallCond.replace({
    10:"Very Excellent",
    9:"Excellent",
    8:"Very Good",
    7:"Good",
    6:"Above Average",
    5:"Average",
    4:"Below Average",
    3:"Fair",
    2:"Poor",
    1:"Very Poor"},inplace=True)

EDA

In [435]:
for i in A.columns:
    if(A[i].dtypes == "object"):
        
        p = A[i].mode()[0]
        A[i] = A[i].fillna(p)
    else:
       
        p = A[i].mean()
        A[i] =A[i].fillna(p)

In [436]:
XT = A.drop(labels=["SalePrice","Id"],axis = 1)
YT = A[["SalePrice"]]

In [438]:
YT.shape

(1460, 1)

Standardizaation and OHE and removing outliers

In [439]:
cat = []
con = []
for i in XT.columns:
    if(XT[i].dtypes == "object"):
        cat.append(i)
    else:
        con.append(i)

In [440]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
from numpy import unique
XT1 = pd.DataFrame(ss.fit_transform(XT[con]),columns=XT[con].columns)
out_index = []
for i in con:
    w = list(XT1[(XT1[i] > 3) | (XT1[i] < -3)].index)
    out_index.extend(w)
outliers = unique(out_index)

XT1 = XT1.drop(labels=outliers, axis = 0)
YT = YT.drop(labels =outliers, axis = 0 )

XT2 = pd.get_dummies(XT[cat])
XT2 = XT2.drop(labels=outliers, axis = 0)
#XT2["MSSubClass_150"] = "0"

Xnew = XT1.join(XT2)
Xnew.shape

(1042, 344)

In [441]:
YT.shape

(1042, 1)

In [442]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,YT,test_size=0.2,random_state=31)

# Full model

In [443]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
model = lr.fit(xtrain,ytrain)
pred_tr = model.predict(xtrain)
pred_ts = model.predict(xtest)

from sklearn.metrics import mean_absolute_error,mean_squared_error
tr_er = round(mean_absolute_error(ytrain,pred_tr),5)
ts_er =  round(mean_absolute_error(ytest,pred_ts),5)

from sklearn.metrics import explained_variance_score
ts_r2 = explained_variance_score(ytest,pred_ts)

from sklearn.metrics import explained_variance_score
tr_r2 = explained_variance_score(ytrain,pred_tr)

print("Training error:",tr_er)
print("Testing error:",ts_er)
print("Testing R2:",ts_r2)
print("Training R2:",tr_r2)

Training error: 9827.47385
Testing error: 12963488478207.756
Testing R2: -1.2251135321835208e+18
Training R2: 0.9519671261395218


# Feature selection(Backward)

In [444]:
from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xtrain)
model = ols.fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              SalePrice   R-squared:                       0.952
Model:                            OLS   Adj. R-squared:                  0.932
Method:                 Least Squares   F-statistic:                     46.94
Date:                Thu, 30 Jun 2022   Prob (F-statistic):          5.30e-281
Time:                        10:55:05   Log-Likelihood:                -9153.7
No. Observations:                 833   AIC:                         1.880e+04
Df Residuals:                     585   BIC:                         1.998e+04
Df Model:                         247                                         
Covariance Type:            nonrobust                                         
==============================================================================================
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
LotFrontage                -1599.6653   1326.431     -1.206      0.228   -4204.812    1005.482
LotArea                     1.197e+04   3255.629      3.677      0.000    5575.613    1.84e+04
YearBuilt                   1.181e+04   3386.988      3.486      0.001    5156.415    1.85e+04
YearRemodAdd                1000.7068   1424.107      0.703      0.483   -1796.279    3797.692
MasVnrArea                   -75.4542   1763.042     -0.043      0.966   -3538.117    3387.209
BsmtFinSF1                  7194.9542   1355.227      5.309      0.000    4533.251    9856.658
BsmtFinSF2                  2088.6345   3157.291      0.662      0.509   -4112.372    8289.641
BsmtUnfSF                   1754.3769   1243.086      1.411      0.159    -687.078    4195.832
TotalBsmtSF                 1.002e+04   1953.053      5.128      0.000    6179.399    1.39e+04
1stFlrSF                    4737.8928   2134.589      2.220      0.027     545.501    8930.285
2ndFlrSF                    1.488e+04   2236.403      6.654      0.000    1.05e+04    1.93e+04
LowQualFinSF                -704.6273   7657.310     -0.092      0.927   -1.57e+04    1.43e+04
GrLivArea                   1.578e+04   1643.957      9.601      0.000    1.26e+04     1.9e+04
BsmtFullBath                 762.9403   1119.133      0.682      0.496   -1435.067    2960.948
BsmtHalfBath               -3541.9454    580.801     -6.098      0.000   -4682.654   -2401.237
FullBath                    2202.7851   1525.697      1.444      0.149    -793.725    5199.296
HalfBath                     303.9962   1287.926      0.236      0.813   -2225.525    2833.517
BedroomAbvGr                -730.8761   1449.876     -0.504      0.614   -3578.473    2116.721
KitchenAbvGr               -3106.9187    509.466     -6.098      0.000   -4107.524   -2106.314
TotRmsAbvGrd                -109.3893   1875.019     -0.058      0.953   -3791.977    3573.199
Fireplaces                   918.0515   2230.963      0.412      0.681   -3463.620    5299.723
GarageYrBlt                -1540.9745   1697.121     -0.908      0.364   -4874.166    1792.217
GarageCars                  3778.5407   1889.413      2.000      0.046      67.682    7489.399
GarageArea                  3588.7778   1881.130      1.908      0.057    -105.812    7283.368
WoodDeckSF                  3646.6899    901.059      4.047      0.000    1876.985    5416.395
OpenPorchSF                 2708.1269   1064.460      2.544      0.011     617.499    4798.755
EnclosedPorch               1037.6610   1143.870      0.907      0.365   -1208.931    3284.253
3SsnPorch                  -1.737e+04   2.42e+04     -0.717      0.474    -6.5e+04    3.02e+04
ScreenPorch                 2361.1760   1366.646      1.728      0.085    -322.954    5045.306
PoolArea                   -1009.2980    16

In [445]:
round(model.rsquared_adj,4)

0.9317

In [446]:
model.pvalues.sort_values().index

Index(['GrLivArea', '2ndFlrSF', 'KitchenAbvGr', 'PoolArea', 'PoolQC_NoPool',
       'Utilities_AllPub', 'BsmtHalfBath', 'BsmtFinSF1', 'TotalBsmtSF',
       'OverallQual_Excellent',
       ...
       'HeatingQC_Po', 'Functional_Sev', 'GarageType_2Types', 'PoolQC_Ex',
       'PoolQC_Fa', 'PoolQC_Gd', 'MiscFeature_Gar2', 'MiscFeature_Othr',
       'MiscFeature_TenC', 'SaleType_Con'],
      dtype='object', length=344)

In [447]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [448]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9317

In [449]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [450]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9317

In [451]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [452]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9317

In [453]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [454]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9317

In [455]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [456]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9317

In [457]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [458]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9317

In [459]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [460]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9317

In [461]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [462]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9317

In [463]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [464]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9317

In [465]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [466]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9317

In [467]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [468]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9317

In [469]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [470]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9317

In [471]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [472]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9317

In [473]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [474]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9317

In [475]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [476]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9317

In [477]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [478]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9317

In [479]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [480]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9318

In [481]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [482]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9318

In [483]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [484]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9318

In [485]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [486]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9318

In [487]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [488]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9318

In [489]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [490]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9318

In [491]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [492]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9318

In [493]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [494]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9318

In [495]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [496]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9318

In [497]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [498]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9318

In [499]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [500]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9318

In [501]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [502]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9318

In [503]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [504]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9318

In [505]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [506]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9318

In [507]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [508]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9318

In [509]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [510]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9318

In [511]:
col_to_drop = list(model.pvalues.sort_values().index)[-2]
col_to_keep = xtrain.columns

In [512]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9318

In [513]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [514]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9318

In [515]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [516]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9319

In [517]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [518]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9319

In [519]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [520]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9319

In [521]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [522]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9319

In [523]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [524]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9319

In [525]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [526]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.932

In [527]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [528]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9321

In [529]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [530]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9321

In [531]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [532]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9321

In [533]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [534]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9321

In [535]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [536]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9321

In [537]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [538]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9321

In [539]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [540]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9321

In [541]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [542]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9321

In [543]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [544]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9323

In [545]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [546]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9323

In [547]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [548]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9323

In [549]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [550]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9323

In [551]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [552]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9323

In [553]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [554]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9323

In [555]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [556]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9323

In [557]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [558]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9323

In [559]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [560]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9323

In [561]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [562]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9323

In [563]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [564]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9324

In [565]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [566]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9324

In [567]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [568]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9324

In [569]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [570]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9324

In [571]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [572]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9325

In [573]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [574]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9325

In [575]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [576]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9326

In [577]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [578]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9326

In [579]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [580]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9327

In [581]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [582]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9328

In [583]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [584]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9328

In [585]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [586]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9329

In [587]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [588]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.933

In [589]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [590]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9332

In [591]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [592]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9332

In [593]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [594]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9332

In [595]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [596]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9332

In [597]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [598]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9332

In [599]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [600]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9333

In [601]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [602]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9334

In [603]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [604]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9334

In [605]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [606]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9335

In [607]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [608]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9336

In [609]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [610]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9337

In [611]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [612]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9337

In [613]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [614]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9338

In [615]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [616]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9339

In [617]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [618]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.934

In [619]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [620]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.934

In [621]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [622]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9341

In [623]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [624]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9341

In [625]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [626]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9342

In [627]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [628]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9343

In [629]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [630]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9344

In [631]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [632]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9345

In [633]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [634]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9346

In [635]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [636]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9347

In [637]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [638]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9348

In [639]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [640]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9349

In [641]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [642]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.935

In [643]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [644]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9351

In [645]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [646]:
col_to_drop

'MSSubClass_30'

In [647]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9352

In [648]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [649]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9353

In [650]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [651]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9353

In [652]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [653]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9354

In [654]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [655]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9354

In [656]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [657]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9354

In [658]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [659]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9355

In [660]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [661]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9356

In [662]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [663]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9357

In [664]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [665]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9358

In [666]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [667]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9358

In [668]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [669]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9359

In [670]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [671]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.936

In [672]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [673]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.936

In [674]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [675]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9361

In [676]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [677]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9362

In [678]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [679]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9362

In [680]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [681]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9363

In [682]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [683]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9364

In [684]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [685]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9364

In [686]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [687]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9365

In [688]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [689]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9366

In [690]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [691]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9367

In [692]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns


In [693]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9368

In [694]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [695]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9369

In [696]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [697]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9369

In [698]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [699]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9369

In [700]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [701]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.937

In [702]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [703]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9371

In [704]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [705]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9372

In [706]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [707]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9373

In [708]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [709]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9373

In [710]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [711]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9373

In [712]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [713]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9374

In [714]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [715]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9375

In [716]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [717]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9376

In [718]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [719]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9377

In [720]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [721]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9377

In [722]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [723]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9378

In [724]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [725]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9379

In [726]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [727]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9379

In [728]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [729]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.938

In [730]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [731]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9381

In [732]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [733]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9381

In [734]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [735]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9381

In [736]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [737]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9382

In [738]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [739]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9383

In [740]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [741]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9383

In [742]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [743]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9384

In [744]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [745]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9384

In [746]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [747]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9385

In [748]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [749]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9386

In [750]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [751]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9386

In [752]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [753]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9386

In [754]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [755]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9386

In [756]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [757]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9387

In [758]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [759]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9388

In [760]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [761]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9388

In [762]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [763]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9389

In [764]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [765]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9389

In [766]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [767]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.939

In [768]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [769]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.939

In [770]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [771]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9391

In [772]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [773]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9391

In [774]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [775]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9392

In [776]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [777]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9392

In [778]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [779]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9392

In [780]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [781]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9393

In [782]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [783]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9393

In [784]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [785]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9393

In [786]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [787]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9393

In [788]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [789]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9394

In [790]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [791]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9394

In [792]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [793]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9394

In [794]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [795]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9395

In [796]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [797]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9395

In [798]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [799]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9396

In [800]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [801]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9396

In [802]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [803]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9396

In [804]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [805]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9396

In [806]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [807]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9397

In [808]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [809]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9397

In [810]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [811]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9397

In [812]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [813]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9397

In [814]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [815]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9398

In [816]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [817]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9398

In [818]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [819]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9398

In [820]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [821]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9398

In [822]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [823]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,4)

0.9398

In [824]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [825]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,7)

0.9398562

In [826]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [827]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,7)

0.9398656

In [828]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [829]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,7)

0.9398812

In [830]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [831]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,7)

0.9398958

In [832]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [833]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,7)

0.9399016

In [834]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [835]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,7)

0.9399016

In [836]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [837]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,7)

0.9399071

In [838]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [839]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,7)

0.9399153

In [840]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [841]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,7)

0.939932

In [842]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [843]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,7)

0.9399421

In [844]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [845]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,7)

0.9399552

In [846]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [847]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,7)

0.9399595

In [848]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [849]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,7)

0.940038

In [850]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [851]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,7)

0.9400436

In [852]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [853]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,7)

0.9400473

In [854]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [855]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,7)

0.9400619

In [856]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [857]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,7)

0.9400688

In [858]:
col_to_drop = list(model.pvalues.sort_values().index)[-1]
col_to_keep = xtrain.columns

In [859]:
xtrain = xtrain.drop(labels=col_to_drop,axis=1)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

round(model.rsquared_adj,7)

0.9400648

In [860]:
#final list of features

col_to_keep

Index(['LotFrontage', 'LotArea', 'YearBuilt', 'BsmtFinSF1', 'TotalBsmtSF',
       '1stFlrSF', '2ndFlrSF', 'BsmtHalfBath', 'FullBath', 'KitchenAbvGr',
       ...
       'MiscFeature_None', 'MiscFeature_Shed', 'MoSold_1', 'MoSold_12',
       'MoSold_5', 'SaleType_CWD', 'SaleCondition_AdjLand',
       'SaleCondition_Alloca', 'SaleCondition_Normal',
       'SaleCondition_Partial'],
      dtype='object', length=139)

In [861]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
from numpy import unique
XT1 = pd.DataFrame(ss.fit_transform(XT[con]),columns=XT[con].columns)
out_index = []
for i in con:
    w = list(XT1[(XT1[i] > 3) | (XT1[i] < -3)].index)
    out_index.extend(w)
outliers = unique(out_index)

XT1 = XT1.drop(labels=outliers, axis = 0)
#YT = YT.drop(labels =outliers, axis = 0 )

XT2 = pd.get_dummies(XT[cat])
XT2 = XT2.drop(labels=outliers, axis = 0)


Xnew = XT1.join(XT2)
Xnew = Xnew[col_to_keep]
#Xnew["MSSubClass_150"] = "0"
#Xnew.shape

from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,YT,test_size=0.2,random_state=31)

In [862]:
xtrain.shape

(833, 139)

New model

In [863]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
model = lr.fit(xtrain,ytrain)
pred_tr = model.predict(xtrain)
pred_ts = model.predict(xtest)

from sklearn.metrics import mean_absolute_error,mean_squared_error
tr_er = round(mean_absolute_error(ytrain,pred_tr),5)
ts_er =  round(mean_absolute_error(ytest,pred_ts),5)

from sklearn.metrics import explained_variance_score
ts_r2 = explained_variance_score(ytest,pred_ts)

from sklearn.metrics import explained_variance_score
tr_r2 = explained_variance_score(ytrain,pred_tr)

print("Training error:",tr_er)
print("Testing error:",ts_er)
print("Testing R2:",ts_r2)
print("Training R2:",tr_r2)

Training error: 10241.0024
Testing error: 12785.11005
Testing R2: 0.9257613497429427
Training R2: 0.9489324497669477


Removing overfitting

Ridge

In [867]:
w=[]
q=0.57
for i in range(0,1000,1):
    q = q+0.0001
    q=round(q,5)
    w.append(q)
    
for i in w:
    
    from sklearn.linear_model import Ridge
    rr = Ridge(alpha=i)
    model = rr.fit(xtrain,ytrain)

    pred_tr = model.predict(xtrain)
    pred_ts = model.predict(xtest)
    tr_er = round(mean_absolute_error(ytrain,pred_tr),5)
    ts_er =  round(mean_absolute_error(ytest,pred_ts),5)
    print("====",i,"====")
    print(tr_er)
    print(ts_er)

==== 0.5701 ====
10309.25111
12555.21896
==== 0.5702 ====
10309.26779
12555.21893
==== 0.5703 ====
10309.28447
12555.21891
==== 0.5704 ====
10309.30114
12555.21888
==== 0.5705 ====
10309.31782
12555.21885
==== 0.5706 ====
10309.33449
12555.21883
==== 0.5707 ====
10309.35116
12555.21881
==== 0.5708 ====
10309.36783
12555.21879
==== 0.5709 ====
10309.3845
12555.21877
==== 0.571 ====
10309.40116
12555.21876
==== 0.5711 ====
10309.41782
12555.21875
==== 0.5712 ====
10309.43448
12555.21874
==== 0.5713 ====
10309.45114
12555.21873
==== 0.5714 ====
10309.46779
12555.21872
==== 0.5715 ====
10309.48445
12555.21872
==== 0.5716 ====
10309.5011
12555.21871
==== 0.5717 ====
10309.51775
12555.21871
==== 0.5718 ====
10309.5344
12555.21871
==== 0.5719 ====
10309.55104
12555.21872
==== 0.572 ====
10309.56769
12555.21872
==== 0.5721 ====
10309.58433
12555.21873
==== 0.5722 ====
10309.60097
12555.21874
==== 0.5723 ====
10309.6176
12555.21875
==== 0.5724 ====
10309.63424
12555.21876
==== 0.5725 ====
10309

==== 0.5904 ====
10312.60885
12555.25252
==== 0.5905 ====
10312.62537
12555.25288
==== 0.5906 ====
10312.64189
12555.25324
==== 0.5907 ====
10312.65841
12555.2536
==== 0.5908 ====
10312.67492
12555.25396
==== 0.5909 ====
10312.69144
12555.25433
==== 0.591 ====
10312.70795
12555.25469
==== 0.5911 ====
10312.72445
12555.25506
==== 0.5912 ====
10312.74096
12555.25543
==== 0.5913 ====
10312.75746
12555.2558
==== 0.5914 ====
10312.77396
12555.25617
==== 0.5915 ====
10312.79046
12555.25655
==== 0.5916 ====
10312.80696
12555.25692
==== 0.5917 ====
10312.82346
12555.2573
==== 0.5918 ====
10312.83995
12555.25768
==== 0.5919 ====
10312.85644
12555.25806
==== 0.592 ====
10312.87293
12555.25845
==== 0.5921 ====
10312.88942
12555.25883
==== 0.5922 ====
10312.90591
12555.25922
==== 0.5923 ====
10312.92239
12555.25961
==== 0.5924 ====
10312.93887
12555.26
==== 0.5925 ====
10312.95535
12555.26039
==== 0.5926 ====
10312.97183
12555.26078
==== 0.5927 ====
10312.9883
12555.26118
==== 0.5928 ====
10313.00

==== 0.6107 ====
10315.92137
12555.36075
==== 0.6108 ====
10315.93749
12555.36146
==== 0.6109 ====
10315.9536
12555.36217
==== 0.611 ====
10315.96971
12555.36288
==== 0.6111 ====
10315.98582
12555.36359
==== 0.6112 ====
10316.00193
12555.3643
==== 0.6113 ====
10316.01804
12555.36501
==== 0.6114 ====
10316.03414
12555.36573
==== 0.6115 ====
10316.05025
12555.36644
==== 0.6116 ====
10316.06635
12555.36716
==== 0.6117 ====
10316.08245
12555.36788
==== 0.6118 ====
10316.09854
12555.3686
==== 0.6119 ====
10316.11464
12555.36933
==== 0.612 ====
10316.13073
12555.37005
==== 0.6121 ====
10316.14682
12555.37078
==== 0.6122 ====
10316.16291
12555.37151
==== 0.6123 ====
10316.179
12555.37224
==== 0.6124 ====
10316.19508
12555.37297
==== 0.6125 ====
10316.21117
12555.3737
==== 0.6126 ====
10316.22725
12555.37444
==== 0.6127 ====
10316.24333
12555.37517
==== 0.6128 ====
10316.25941
12555.3768
==== 0.6129 ====
10316.27548
12555.38514
==== 0.613 ====
10316.29155
12555.39349
==== 0.6131 ====
10316.307

==== 0.6313 ====
10319.20077
12556.92784
==== 0.6314 ====
10319.21649
12556.93626
==== 0.6315 ====
10319.23221
12556.94468
==== 0.6316 ====
10319.24793
12556.9531
==== 0.6317 ====
10319.26365
12556.96153
==== 0.6318 ====
10319.27937
12556.96995
==== 0.6319 ====
10319.29508
12556.97838
==== 0.632 ====
10319.31079
12556.9868
==== 0.6321 ====
10319.3265
12556.99523
==== 0.6322 ====
10319.34221
12557.00365
==== 0.6323 ====
10319.35792
12557.01208
==== 0.6324 ====
10319.37362
12557.0205
==== 0.6325 ====
10319.38933
12557.02893
==== 0.6326 ====
10319.40503
12557.03736
==== 0.6327 ====
10319.42073
12557.04578
==== 0.6328 ====
10319.43642
12557.05421
==== 0.6329 ====
10319.45212
12557.06264
==== 0.633 ====
10319.46781
12557.07107
==== 0.6331 ====
10319.4835
12557.0795
==== 0.6332 ====
10319.49919
12557.08792
==== 0.6333 ====
10319.51488
12557.09635
==== 0.6334 ====
10319.53057
12557.10478
==== 0.6335 ====
10319.54625
12557.11321
==== 0.6336 ====
10319.56193
12557.12164
==== 0.6337 ====
10319.5

==== 0.6523 ====
10322.47746
12558.80276
==== 0.6524 ====
10322.49298
12558.81417
==== 0.6525 ====
10322.50851
12558.82557
==== 0.6526 ====
10322.52403
12558.83698
==== 0.6527 ====
10322.53955
12558.84838
==== 0.6528 ====
10322.55507
12558.85979
==== 0.6529 ====
10322.57058
12558.87119
==== 0.653 ====
10322.5861
12558.8826
==== 0.6531 ====
10322.60161
12558.894
==== 0.6532 ====
10322.61712
12558.90541
==== 0.6533 ====
10322.63263
12558.91682
==== 0.6534 ====
10322.64814
12558.92822
==== 0.6535 ====
10322.66365
12558.93963
==== 0.6536 ====
10322.67915
12558.95103
==== 0.6537 ====
10322.69465
12558.96244
==== 0.6538 ====
10322.71015
12558.97385
==== 0.6539 ====
10322.72565
12558.98525
==== 0.654 ====
10322.74115
12558.99666
==== 0.6541 ====
10322.75664
12559.00807
==== 0.6542 ====
10322.77214
12559.01947
==== 0.6543 ====
10322.78763
12559.03088
==== 0.6544 ====
10322.80312
12559.04229
==== 0.6545 ====
10322.8186
12559.0537
==== 0.6546 ====
10322.83409
12559.0651
==== 0.6547 ====
10322.84

In [869]:
    from sklearn.linear_model import Ridge
    rr = Ridge(alpha=0.5716)
    model = rr.fit(xtrain,ytrain)

    pred_tr = model.predict(xtrain)
    pred_ts = model.predict(xtest)
    tr_er = round(mean_absolute_error(ytrain,pred_tr),5)
    ts_er =  round(mean_absolute_error(ytest,pred_ts),5)
    print("========")
    print(tr_er)
    print(ts_er)

10309.5011
12555.21871


Lasso

In [873]:
w=[]
q=20.06 
for i in range(0,1000,1):
    q = q+0.0001
    q=round(q,5)
    w.append(q)
    
for i in w:
    
    from sklearn.linear_model import Lasso
    ls = Lasso(alpha=i)
    model = ls.fit(xtrain,ytrain)

    pred_tr = model.predict(xtrain)
    pred_ts = model.predict(xtest)
    tr_er = round(mean_absolute_error(ytrain,pred_tr),5)
    ts_er =  round(mean_absolute_error(ytest,pred_ts),5)
    print("====",i,"====")
    print(tr_er)
    print(ts_er)

==== 20.0601 ====
10449.35315
12289.07425
==== 20.0602 ====
10449.35487
12289.07421
==== 20.0603 ====
10449.35658
12289.07417
==== 20.0604 ====
10449.3583
12289.07413
==== 20.0605 ====
10449.36001
12289.07409
==== 20.0606 ====
10449.36173
12289.07405
==== 20.0607 ====
10449.36344
12289.07401
==== 20.0608 ====
10449.36516
12289.07397
==== 20.0609 ====
10449.36687
12289.07393
==== 20.061 ====
10449.36859
12289.07389
==== 20.0611 ====
10449.3703
12289.07385
==== 20.0612 ====
10449.37202
12289.07381
==== 20.0613 ====
10449.37373
12289.07377
==== 20.0614 ====
10449.37545
12289.07373
==== 20.0615 ====
10449.37716
12289.07369
==== 20.0616 ====
10449.37888
12289.07365
==== 20.0617 ====
10449.38059
12289.07361
==== 20.0618 ====
10449.38231
12289.07357
==== 20.0619 ====
10449.38402
12289.07353
==== 20.062 ====
10449.38574
12289.07349
==== 20.0621 ====
10449.38745
12289.07345
==== 20.0622 ====
10449.38917
12289.07341
==== 20.0623 ====
10449.39088
12289.07337
==== 20.0624 ====
10449.3926
12289.073

==== 20.0798 ====
10449.691
12289.09255
==== 20.0799 ====
10449.69271
12289.0927
==== 20.08 ====
10449.69443
12289.09285
==== 20.0801 ====
10449.69614
12289.093
==== 20.0802 ====
10449.69786
12289.09315
==== 20.0803 ====
10449.69957
12289.0933
==== 20.0804 ====
10449.70129
12289.09346
==== 20.0805 ====
10449.703
12289.09361
==== 20.0806 ====
10449.70472
12289.09376
==== 20.0807 ====
10449.70643
12289.09391
==== 20.0808 ====
10449.70815
12289.09406
==== 20.0809 ====
10449.70986
12289.09421
==== 20.081 ====
10449.71158
12289.09437
==== 20.0811 ====
10449.71329
12289.09452
==== 20.0812 ====
10449.71501
12289.09467
==== 20.0813 ====
10449.71672
12289.09482
==== 20.0814 ====
10449.71844
12289.09497
==== 20.0815 ====
10449.72015
12289.09512
==== 20.0816 ====
10449.72187
12289.09527
==== 20.0817 ====
10449.72358
12289.09543
==== 20.0818 ====
10449.7253
12289.09558
==== 20.0819 ====
10449.72701
12289.09573
==== 20.082 ====
10449.72873
12289.09588
==== 20.0821 ====
10449.73044
12289.09603
==== 

==== 20.0998 ====
10450.03399
12289.12286
==== 20.0999 ====
10450.03571
12289.12301
==== 20.1 ====
10450.03742
12289.12316
==== 20.1001 ====
10450.03914
12289.12331
==== 20.1002 ====
10450.04085
12289.12346
==== 20.1003 ====
10450.04257
12289.12361
==== 20.1004 ====
10450.04428
12289.12377
==== 20.1005 ====
10450.046
12289.12392
==== 20.1006 ====
10450.04771
12289.12407
==== 20.1007 ====
10450.04943
12289.12422
==== 20.1008 ====
10450.05114
12289.12437
==== 20.1009 ====
10450.05286
12289.12452
==== 20.101 ====
10450.05457
12289.12467
==== 20.1011 ====
10450.05629
12289.12483
==== 20.1012 ====
10450.058
12289.12498
==== 20.1013 ====
10450.05972
12289.12513
==== 20.1014 ====
10450.06143
12289.12528
==== 20.1015 ====
10450.06315
12289.12543
==== 20.1016 ====
10450.06486
12289.12558
==== 20.1017 ====
10450.06658
12289.12574
==== 20.1018 ====
10450.06829
12289.12589
==== 20.1019 ====
10450.07001
12289.12604
==== 20.102 ====
10450.07172
12289.12619
==== 20.1021 ====
10450.07344
12289.12634
=

==== 20.1195 ====
10450.37184
12289.15271
==== 20.1196 ====
10450.37355
12289.15286
==== 20.1197 ====
10450.37527
12289.15301
==== 20.1198 ====
10450.37698
12289.15317
==== 20.1199 ====
10450.3787
12289.15332
==== 20.12 ====
10450.38041
12289.15347
==== 20.1201 ====
10450.38213
12289.15362
==== 20.1202 ====
10450.38384
12289.15377
==== 20.1203 ====
10450.38556
12289.15392
==== 20.1204 ====
10450.38727
12289.15407
==== 20.1205 ====
10450.38899
12289.15423
==== 20.1206 ====
10450.3907
12289.15438
==== 20.1207 ====
10450.39242
12289.15453
==== 20.1208 ====
10450.39413
12289.15468
==== 20.1209 ====
10450.39585
12289.15483
==== 20.121 ====
10450.39756
12289.15498
==== 20.1211 ====
10450.39928
12289.15514
==== 20.1212 ====
10450.40099
12289.15529
==== 20.1213 ====
10450.40271
12289.15544
==== 20.1214 ====
10450.40442
12289.15559
==== 20.1215 ====
10450.40614
12289.15574
==== 20.1216 ====
10450.40785
12289.15589
==== 20.1217 ====
10450.40957
12289.15604
==== 20.1218 ====
10450.41128
12289.156

==== 20.1394 ====
10450.71311
12289.18287
==== 20.1395 ====
10450.71483
12289.18302
==== 20.1396 ====
10450.71654
12289.18317
==== 20.1397 ====
10450.71826
12289.18332
==== 20.1398 ====
10450.71997
12289.18347
==== 20.1399 ====
10450.72169
12289.18363
==== 20.14 ====
10450.7234
12289.18378
==== 20.1401 ====
10450.72512
12289.18393
==== 20.1402 ====
10450.72683
12289.18408
==== 20.1403 ====
10450.72855
12289.18423
==== 20.1404 ====
10450.73026
12289.18438
==== 20.1405 ====
10450.73198
12289.18454
==== 20.1406 ====
10450.73369
12289.18469
==== 20.1407 ====
10450.73541
12289.18484
==== 20.1408 ====
10450.73712
12289.18499
==== 20.1409 ====
10450.73884
12289.18514
==== 20.141 ====
10450.74055
12289.18529
==== 20.1411 ====
10450.74227
12289.18544
==== 20.1412 ====
10450.74398
12289.1856
==== 20.1413 ====
10450.7457
12289.18575
==== 20.1414 ====
10450.74741
12289.1859
==== 20.1415 ====
10450.74913
12289.18605
==== 20.1416 ====
10450.75084
12289.1862
==== 20.1417 ====
10450.75256
12289.18635


==== 20.1594 ====
10451.05611
12289.21318
==== 20.1595 ====
10451.05782
12289.21333
==== 20.1596 ====
10451.05954
12289.21348
==== 20.1597 ====
10451.06125
12289.21363
==== 20.1598 ====
10451.06297
12289.21378
==== 20.1599 ====
10451.06468
12289.21394
==== 20.16 ====
10451.0664
12289.21409


In [874]:
    from sklearn.linear_model import Lasso
    ls = Lasso(alpha=20.0661)
    model = ls.fit(xtrain,ytrain)

    pred_tr = model.predict(xtrain)
    pred_ts = model.predict(xtest)
    tr_er = round(mean_absolute_error(ytrain,pred_tr),5)
    ts_er =  round(mean_absolute_error(ytest,pred_ts),5)
    print("========")
    print(tr_er)
    print(ts_er)

10449.45605
12289.07186


# best model

In [877]:
    from sklearn.linear_model import Lasso
    ls = Lasso(alpha=20.0661)
    model = ls.fit(xtrain,ytrain)

    pred_tr = model.predict(xtrain)
    pred_ts = model.predict(xtest)
    tr_er = round(mean_absolute_error(ytrain,pred_tr),5)
    ts_er =  round(mean_absolute_error(ytest,pred_ts),5)
    print("========")
    print(tr_er)
    print(ts_er)
    ts_r2 = explained_variance_score(ytest,pred_ts)
    tr_r2 = explained_variance_score(ytrain,pred_tr)
    print(tr_r2)
    print(ts_r2)

10449.45605
12289.07186
0.9469969746856525
0.9310894180123207


# Testing Data

In [878]:
B = pd.read_csv("D:/EtlHiveDs/Project/Project1/testing_set.csv")

In [879]:
B["MSSubClass"] =B["MSSubClass"].map(str)
B["MoSold"] = B["MoSold"].map(str)

In [880]:
B["Alley"] = B["Alley"].fillna("NoAlley")
B["BsmtQual"] = B["BsmtQual"].fillna("NoBasement")
B["BsmtCond"] = B["BsmtCond"].fillna("NoBasement")
B["BsmtExposure"] = B["BsmtExposure"].fillna("NoBasement")
B["BsmtFinType1"] = B["BsmtFinType1"].fillna("NoBasement")
B["BsmtFinType2"] = B["BsmtFinType2"].fillna("NoBasement")
B["FireplaceQu"] = B["FireplaceQu"].fillna("NoFireplace")
B["GarageType"] = B["GarageType"].fillna("NoGarage")
B["GarageFinish"] = B["GarageFinish"].fillna("NoGarage")
B["GarageQual"] = B["GarageQual"].fillna("NoGarage")
B["GarageCond"] = B["GarageCond"].fillna("NoGarage")
B["PoolQC"] = B["PoolQC"].fillna("NoPool")
B["Fence"] = B["Fence"].fillna("NoFence")
B["MiscFeature"] =B["MiscFeature"].fillna("None")

In [881]:
B.OverallQual.replace({
    10:"Very Excellent",
    9:"Excellent",
    8:"Very Good",
    7:"Good",
    6:"Above Average",
    5:"Average",
    4:"Below Average",
    3:"Fair",
    2:"Poor",
    1:"Very Poor",
    0:""},inplace=True)

In [882]:
B.OverallCond.replace({
    10:"Very Excellent",
    9:"Excellent",
    8:"Very Good",
    7:"Good",
    6:"Above Average",
    5:"Average",
    4:"Below Average",
    3:"Fair",
    2:"Poor",
    1:"Very Poor"},inplace=True)

In [883]:
for i in B.columns:
    if(B[i].dtypes == "object"):
        
        q = B[i].mode()[0]
        B[i] = B[i].fillna(q)
    else:
       
        q = B[i].mean()
        B[i] =B[i].fillna(q)

In [884]:
XTe = B.drop(labels=["Id"],axis = 1)


In [885]:
Te = XTe[con]

In [886]:

XTe1 = pd.DataFrame(ss.transform(Te),columns=Te.columns)
#ss.transform
#XTe1 = pd.DataFrame(ss.fit_transform(XTe[con]),columns=XTe[con].columns)
#out_index = []
#for i in con:
    #w = list(XTe1[(XTe1[i] > 3) | (XTe1[i] < -3)].index)
    #out_index.extend(w)
#outliers = unique(out_index)
#XTe1 = XTe1.drop(labels=outliers, axis = 0)
#YT = YT.drop(labels =outliers, axis = 0 )

XTe2 = pd.get_dummies(XTe[cat])
#XTe2 = XTe2.drop(labels=outliers, axis = 0)

XTenew = XTe1.join(XTe2)
XTenew.shape

(1459, 327)

In [887]:
col_to_keep

Index(['LotFrontage', 'LotArea', 'YearBuilt', 'BsmtFinSF1', 'TotalBsmtSF',
       '1stFlrSF', '2ndFlrSF', 'BsmtHalfBath', 'FullBath', 'KitchenAbvGr',
       ...
       'MiscFeature_None', 'MiscFeature_Shed', 'MoSold_1', 'MoSold_12',
       'MoSold_5', 'SaleType_CWD', 'SaleCondition_AdjLand',
       'SaleCondition_Alloca', 'SaleCondition_Normal',
       'SaleCondition_Partial'],
      dtype='object', length=139)

In [888]:
XTeFinal = XTenew[col_to_keep]

In [895]:
    from sklearn.linear_model import Lasso
    ls = Lasso(alpha=20.0661)
    model = ls.fit(xtrain,ytrain)

    pred_tr = model.predict(xtrain)
    pred_ts = model.predict(xtest)
    tr_er = round(mean_absolute_error(ytrain,pred_tr),5)
    ts_er =  round(mean_absolute_error(ytest,pred_ts),5)
    print("========")
    print("Training error :",tr_er)
    print("Testing error :",ts_er)
    ts_r2 = explained_variance_score(ytest,pred_ts)
    tr_r2 = explained_variance_score(ytrain,pred_tr)
    print("Training r2_score",tr_r2)
    print("Testing r2_score :",ts_r2)

Training error : 10449.45605
Testing error : 12289.07186
Training r2_score 0.9469969746856525
Testing r2_score : 0.9310894180123207


In [896]:
XTeFinal.shape

(1459, 139)

Final predictions

In [897]:
testing_set_pred = model.predict(XTeFinal)

In [898]:
P = pd.DataFrame(list(testing_set_pred), columns=["PredictedSalePrice"])
I = B[["Id"]]

In [899]:
Soln = I.join(P)

In [900]:
Soln.to_csv("SolutionFinal.csv",index=False)